# Notebook to sort RER files into task->semester->item->file format

the inputs to this notebook are all RER files as of march 2024 (rer_missing) and the csv file with corresponding metadata for all of these files from Nuria 'rer_audios_descr.csv'

In [1]:
# import dependencies
import os
import shutil
import pandas as pd

In [35]:
#load csv from Nuria into pandas dataframe
    data=pd.read_csv('rer_audios_descr.csv')
    #remove files that have NaN in the subjects column (their metadata is missing, these are mostly the 'test' junk files)
    data.dropna(subset=['Student Tracking Id'], inplace=True) 
# fix the inconsistently named task names
# remove "(" and everything after it
data['Task Name'] = data['Task Name'].str.split('(').str[0]
# remove "-" and everything after it
data['Task Name'] = data['Task Name'].str.split('-').str[0]
# remove ":" and everything after it
data['Task Name'] = data['Task Name'].str.split(':').str[0]
# remove leading and trailing spaces
data['Task Name'] = data['Task Name'].str.strip()
# remove z typo, fix PAL spellings
data['Task Name'] = data['Task Name'].str.replace('z', '').str.replace('Paired Assoc. Learning', 'Paired Associate Learning').str.replace('Associates', 'Associate')


/tmp/ipykernel_1657847/3887735639.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Task Name'] = data['Task Name'].str.replace('z', '').str.replace('Paired Assoc. Learning', 'Paired Associate Learning').str.replace('Associates', 'Associate')


In [56]:
#look at the dataframe to see if it looks good
data

,Unnamed: 0,0,1,2,Audio Response,Assessment Name,Task Name,Form Name,Item Name,Student Tracking Id,Grade,semester
14,14,rer_missing,20181024,G0_F1_D1-Deletion(KF1)-DEL_0_S_0003_a_D-11_18_...,G0_F1_D1-Deletion(KF1)-DEL_0_S_0003_a_D-11_18_...,G0_F1_D1,Deletion,z,DEL_0_S_0003_a_D,11_18_0_0002,KG,Y1_fall
15,15,rer_missing,20181024,G0_F1_D1-Deletion(KF1)-DEL_0_S_0002_a_D-11_18_...,G0_F1_D1-Deletion(KF1)-DEL_0_S_0002_a_D-11_18_...,G0_F1_D1,Deletion,z,DEL_0_S_0002_a_D,11_18_0_0002,KG,Y1_fall
16,16,rer_missing,20181024,G0_F1_D1-Deletion(KF1)-DEL_0_S_0001_b_U-11_18_...,G0_F1_D1-Deletion(KF1)-DEL_0_S_0001_b_U-11_18_...,G0_F1_D1,Deletion,z,DEL_0_S_0001_b_U,11_18_0_0001,KG,Y1_fall
17,17,rer_missing,20181024,G0_F1_D1-Deletion(KF1)-DEL_0_S_0001_b_U-11_18_...,G0_F1_D1-Deletion(KF1)-DEL_0_S_0001_b_U-11_18_...,G0_F1_D1,Deletion,z,DEL_0_S_0001_b_U,11_18_0_0002,KG,Y1_fall
18,18,rer_missing,20181024,G0_F1_D1-Deletion(KF1)-DEL_0_S_0001_b_U-11_18_...,G0_F1_D1-Deletion(KF1)-DEL_0_S_0001_b_U-11_18_...,G0_F1_D1,Deletion,z,DEL_0_S_0001_b_U,11_18_0_0011,KG,Y1_fall
...,...,...,...,...,...,...,...,...,...,...,...,...
513382,513382,rer_missing,SCORE_MISSING,G0_F1_D1-Synonyms(KF1)-SYN_0_S_0012_a_U-11_18_...,G0_F1_D1-Synonyms(KF1)-SYN_0_S_0012_a_U-11_18_...,G0_F1_D1,Synonyms,z,SYN_0_S_0012_a_U,11_18_0_0062,KG,Y1_fall
513384,513384,rer_missing,SCORE_MISSING,G0_F1_D1-Synonyms(KF1)-SYN_0_S_0014_a_U-11_18_...,G0_F1_D1-Synonyms(KF1)-SYN_0_S_0014_a_U-11_18_...,G0_F1_D1,Synonyms,z,SYN_0_S_0014_a_U,11_18_0_0062,KG,Y1_fall
513385,513385,rer_missing,SCORE_MISSING,G0_F1_D1-Synonyms(KF1)-SYN_0_S_0018_a_U-11_18_...,G0_F1_D1-Synonyms(KF1)-SYN_0_S_0018_a_U-11_18_...,G0_F1_D1,Synonyms,z,SYN_0_S_0018_a_U,11_18_0_0062,KG,Y1_fall
513386,513386,rer_missing,SCORE_MISSING,G0_F1_D1-Synonyms(KF1)-SYN_0_S_0022_a_U-11_18_...,G0_F1_D1-Synonyms(KF1)-SYN_0_S_0022_a_U-11_18_...,G0_F1_D1,Synonyms,z,SYN_0_S_0022_a_U,11_18_0_0062,KG,Y1_fall


In [39]:
#print all of the unique tasks and make sure they look right
unique_tasks = data['Task Name'].unique()
print(sorted(unique_tasks))

['Blending', 'CELF_RS', 'Child Verbal Assent Form', 'Deletion', 'Digit Span', 'EVT', 'Growth Mindset Question', 'KTEA LWR', 'KTEA NWD', 'Letter Naming', 'Letter Naming Context', 'Letter Naming Isolation', 'Letter Sounds', 'Letter Sounds Context', 'Letter Sounds Isolated', 'Nonword Reading', 'Nonword Repetition', 'Oral Reading Fluency', 'Paired Associate Learning', 'Parent Technology Survey', 'Pearson Copyright', 'Picture Naming', 'Rapid Letter Naming', 'Rapid Naming', 'Rapid Object Naming', 'Sentence Repetition', 'Set for Variability', 'Synonyms', 'Visual Attention', 'WPPSI PN', 'Word Reading']


In [43]:
# 6 of the tasks do not need item level subfolders. make a dataframe that incluedes these and another that excludes these
tasks_to_exclude = ['Letter Naming Context', 'Letter Naming Isolation', 'Letter Sounds Context', 
                    'Letter Sounds Isolated', 'Rapid Letter Naming', 'Rapid Object Naming']
# Dataframe with rows where 'Task Name' is not in the list of tasks to exclude
df_standard = data[~data['Task Name'].isin(tasks_to_exclude)]
# Dataframe with rows where 'Task Name' is in the list of tasks to exclude
df_noitem = data[data['Task Name'].isin(tasks_to_exclude)]

In [55]:
#copy the files that do not need to be placed in item specific folders (~7000 files, could take like 20 min)

destination='/nese/mit/group/sig/projects/readnet/rer_sorted'
# Iterate over the DataFrame rows
for index, row in df_noitem.iterrows():
    # Define the destination directory path
    dest_dir = os.path.join(destination, row['Task Name'], row['semester'])
        
    src_file = os.path.join('/nese/mit/group/sig/projects/readnet/rer_missing/', str(row['1']), str(row['2']))

    # Create the destination directory if it doesn't exist
    os.makedirs(dest_dir, exist_ok=True)
    
    # Copy the file
    shutil.copy(src_file, dest_dir)

In [ ]:
#copy the files that DO need to be placed in item specific folders (~400,000 files, could take like... 8-24+ hours depending on the file system etc)
#NOTE this is actually broken out into the python script sort_rer.py now. It will run faster there instead of here.

# destination='/nese/mit/group/sig/projects/readnet/rer_sorted'
# # Iterate over the DataFrame rows
# for index, row in df_standard.iterrows():
#     # Define the destination directory path
#     dest_dir = os.path.join(destination, row['Task Name'], row['semester'], row['Item Name'])
        
#     src_file = os.path.join('/nese/mit/group/sig/projects/readnet/rer_missing/', str(row['1']), str(row['2']))

#     # Create the destination directory if it doesn't exist
#     os.makedirs(dest_dir, exist_ok=True)
    
#     # Copy the file
#     shutil.copy(src_file, dest_dir)

In [2]:
#move all files with nans in the columns (no metadata) to no_metadata folder
data=pd.read_csv('rer_audios_descr.csv')
nan_rows = data[pd.isna(data['Student Tracking Id'])]

/tmp/ipykernel_39121/1040383510.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('rer_audios_descr.csv')


In [8]:
destination='/nese/mit/group/sig/projects/readnet/rer_sorted/no_metadata/'
for index, row in nan_rows.iterrows():
    # Define the destination directory path        
    src_file = os.path.join('/nese/mit/group/sig/projects/readnet/rer_missing/', str(row['1']), str(row['2']))
    
    # Copy the file
    shutil.copy(src_file, destination)

# part 2, some files were not in the csv, move those files from no_metadata to the proper folder

In [2]:
# load the updated csv
data_new=pd.read_csv('rer_audios_descr_corrected_04_25.csv')
#remove files that have NaN in the subjects column (their metadata is missing, these are mostly the 'test' junk files)
data_new.dropna(subset=['Student Tracking Id'], inplace=True) 
# fix the inconsistently named task names
data_new['Task Name'] = data_new['Task Name'].str.replace('ZFall2018 - ', '').str.replace('zFall2018 - ', '')
data_new['Task Name'] = data_new['Task Name'].str.replace('letter naming 6 test', 'Letter Naming').str.replace('letter naming test 5 items', 'Letter Naming').str.replace('Letter naming test 6 items', 'Letter Naming')
data_new['Task Name'] = data_new['Task Name'].str.replace('SRT test - why are options stupid', 'Sentence Repetition')
# remove "(" and everything after it
data_new['Task Name'] = data_new['Task Name'].str.split('(').str[0]
# remove "-" and everything after it
data_new['Task Name'] = data_new['Task Name'].str.split('-').str[0]
# remove ":" and everything after it
data_new['Task Name'] = data_new['Task Name'].str.split(':').str[0]
# remove leading and trailing spaces
data_new['Task Name'] = data_new['Task Name'].str.strip()
# remove z typo, fix PAL spellings
data_new['Task Name'] = data_new['Task Name'].str.replace('z', '').str.replace('Paired Assoc. Learning', 'Paired Associate Learning').str.replace('Associates', 'Associate')

/tmp/ipykernel_1015601/2102771427.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_new=pd.read_csv('rer_audios_descr_corrected_04_25.csv')
/tmp/ipykernel_1015601/2102771427.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  data_new['Task Name'] = data_new['Task Name'].str.replace('z', '').str.replace('Paired Assoc. Learning', 'Paired Associate Learning').str.replace('Associates', 'Associate')


In [3]:
#rename the semesters to match this current convention
mode_mapping = {
    'Fall2018': 'Y1_fall',
    'Fall2019': 'Y2_fall',
    'Fall2020': 'Y3',
    'Fall2021': 'Y4_fall',
    'Fall2022': 'Y5_fall',
    'Winter-Spring2019': 'Y1_winter',
    'Winter-Spring2020': 'Y2_winter',
    'Winter-Spring2021': 'Y3',
    'Winter-Spring2022': 'Y4_winter',
    'Winter-Spring 2023': 'Y5_winter',
    'Winter-Spring2023': 'Y5_winter'
}
# Replace the old semester names with the new ones
data_new['semester'] = data_new['semester'].replace(mode_mapping)

In [4]:
#list all of the wav files in the no_metadata folder and save to dataframe
import os
import glob

path = '/nese/mit/group/sig/projects/readnet/rer_sorted/no_metadata'
extension = 'wav'

os.chdir(path)
result = glob.glob(f'*.{extension}')
no_metadata = pd.DataFrame(result, columns=['Audio Response'])
print(no_metadata)

                                          Audio Response
0      G0_F1_D1-Deletion(KF1)-DEL_0_S_0002_b_P-test_s...
1      G0_F1_D1-Deletion(KF1)-DEL_0_S_0002_b_P-test_s...
2      G0_F1_D1-LetterNaming-Context(K)-RAN_Upper_W-t...
3      G0_F1_D1-LetterNaming-Context(K)-RAN_Upper_W-t...
4      G0_F1_D1-LetterNaming-Context(K)-RAN_Upper_W-t...
...                                                  ...
60336  Winter22VirtualFidelityTestout-Part1-OralReadi...
60337  Winter22VirtualFidelityTestout-Part1-OralReadi...
60338  Winter22VirtualFidelityTestout-Part1-OralReadi...
60339  Winter22VirtualFidelityTestout-Part1-OralReadi...
60340  Winter22VirtualFidelityTestout-Part1-OralReadi...

[60341 rows x 1 columns]


right now there are 60k files in no_metadata

In [5]:
#get the rows of data_new that have the same 'Audio Response' as no_metadata.
#these are the files we will try to move
matching_rows = data_new[data_new['Audio Response'].isin(no_metadata['Audio Response'])]

In [9]:
#copy the no_metadata files in item specific folders (~60k files, could take like... an hour or 2 max ?)

destination='/nese/mit/group/sig/projects/readnet/rer_sorted'
# Iterate over the DataFrame rows
for index, row in matching_rows.iterrows():
    # Define the destination directory path
    dest_dir = os.path.join(destination, row['Task Name'], row['semester'], row['Item Name'])
        
    src_file = os.path.join('/nese/mit/group/sig/projects/readnet/rer_sorted/no_metadata/', str(row['2']))

    #print(src_file, dest_dir)
    # Create the destination directory if it doesn't exist
    os.makedirs(dest_dir, exist_ok=True)
    
    # Copy the file
    shutil.copy(src_file, dest_dir)
    #remove the old file
    os.remove(src_file)

In [10]:
#list all of the wav files in the no_metadata folder afterwards
import os
import glob

path = '/nese/mit/group/sig/projects/readnet/rer_sorted/no_metadata'
extension = 'wav'

os.chdir(path)
result = glob.glob(f'*.{extension}')
post_no_metadata = pd.DataFrame(result, columns=['Audio Response'])
print(post_no_metadata)

                                         Audio Response
0                                638047146367380190.wav
1                                638047146270344700.wav
2                                638046406710939100.wav
3                                638046406661012410.wav
4                                638046406628054460.wav
...                                                 ...
5393  Winter22VirtualFidelityTestout-Part1-OralReadi...
5394  Winter22VirtualFidelityTestout-Part1-OralReadi...
5395  Winter22VirtualFidelityTestout-Part1-OralReadi...
5396  Winter22VirtualFidelityTestout-Part1-OralReadi...
5397  Winter22VirtualFidelityTestout-Part1-OralReadi...

[5398 rows x 1 columns]


 now there are 5k files in no_metadata

# part 3, make a csv file which lists the files of each directory

In [18]:
import os
import csv

# Specify the target directory path
target_directory = "/nese/mit/group/sig/projects/readnet/rer_sorted/"

# Get a list of all subdirectories
subdirectories = [f.path for f in os.scandir(target_directory) if f.is_dir()]
subdirectories = [os.path.basename(os.path.normpath(path)) for path in subdirectories]

with open('all_files_2024-04-29.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for folder_name in subdirectories:
        for root, dirs, files in os.walk(f'{target_directory}/{folder_name}'):
            for file in files:
                writer.writerow([folder_name, file])